In [7]:
import os
import logging
from pathlib import Path
from typing import Optional
from openai import OpenAI


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY', ''))

def build_prompt(meeting_transcript: str) -> str:
    try:
        template = {
            'system_role': "당신은 회의 전체 내용을 카테고리별로 요약해주는 요약 AI입니다.",
            'output_format': [
                "다음 회의 날짜와 시간: OOO",
                "아이템: OOO",
                "해야 할 일: OOO",
                "팀원들의 의견: OOO",
                "멘토의 피드백: OOO"
            ]
        }

        prompt = f""" {template['system_role']}
아래 회의 전체 내용을 다음 형식으로 요약하세요:

[출력 예시]
{chr(10).join(template['output_format'])}

[회의 전체 내용]
{meeting_transcript}

[정리된 결과] """
        return prompt

    except Exception as e:
        logging.error(f"Failed to build prompt: {str(e)}")
        raise

def summarize_meeting(transcript: str):
    prompt = build_prompt(transcript)


    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 회의 전체 내용을 카테고리별로 요약해주는 요약 AI입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
    )


    return response.choices[0].message.content.strip()

# 테스트용
meeting_transcript = """
회의 시작하겠습니다. 모두 접속한 것 같네요. 오늘은 API 리스크 보험 모델 설계와 관련된 내용을 중점적으로 다뤄볼게요. 지난주에 각자 맡은 부분들 어떻게 진행됐는지 공유부터 해볼까요?

데이터 수집부터 얘기할게요. 공공 데이터 API는 대부분 연결했는데, SLA 위반 기준이 정확히 명시된 건 많지 않아요. 특히 중소 규모 API는 관련 데이터가 누락된 게 많더라고요. 장애 기록도 일부는 1년 이상 안 올라온 것도 있었고요.

그럼 그 기준은 어떻게 정리하고 있어요? 가이드라인 같은 거 임의로 세워봤어요?

네. 일단 일일 호출량 기준 1천 건 이상인 API를 '주요 API'로 분류하고, 여기에 장애 빈도랑 지속 시간을 결합해서 등급을 나눴어요. 근데 아직 통계적으로 유의미한 결과는 못 냈고요.

오케이. 그럼 보험 설계 쪽은요?

저는 보험료 책정 모델 초안을 만들어봤어요. 호출량, 장애 확률, 지속 시간 3가지를 주요 변수로 넣어서 회귀 분석 돌려봤는데, 아직 잔차가 꽤 크더라고요. 특히 batch API보다 실시간 API에서 예측 오차가 컸어요.

그럴 수 있어요. 실시간 API는 트래픽이 예측 불가능하니까. 그럼 이걸 구간별로 나눠서 분포를 다시 볼 수 있을까요?

가능하죠. 0~1000, 1000~10000, 10000 이상으로 나눠서 다시 돌려볼게요.

좋아요. 그리고 리스크 시나리오는 어떻게 정리되고 있어요?

실제 장애 사례를 기반으로 3가지 케이스 만들었어요. 1번은 호출량 급증, 2번은 외부 API 장애 전이, 3번은 인증 실패 반복이에요. 여기에 대해 보상 조건을 다르게 적용해봤어요.

그거 괜찮네요. 근데 멘토님이 지난번에 시나리오가 너무 일반적이라고 하셨잖아요. 혹시 최근 핀테크 쪽 이슈 기반으로 리스크 요소 추가할 수 있을까요?

그 부분 제가 찾아볼게요. 실제로 인증 API 장애로 결제 오류난 사건이 몇 건 있더라고요. 사례 정리해서 공유드릴게요.

좋아요. 그럼 다음 회의는 다음 주 화요일 오전 10시로 정할게요. 그때까지 각자 정리된 모델이랑 시나리오 안 가져오시면 회의 길어질 수 있어요!

네 알겠습니다. 정리해서 올리겠습니다.

수고하셨습니다!
"""

if __name__ == "__main__":

    if not os.environ.get('OPENAI_API_KEY'):
        api_key = input("OpenAI API 키를 입력하세요: ")
        os.environ['OPENAI_API_KEY'] = api_key
        client = OpenAI(api_key=api_key)


    try:
        summary = summarize_meeting(meeting_transcript)
        print(summary)
    except Exception as e:
        logging.error(f"요약 중 오류 발생: {str(e)}")


다음 회의 날짜와 시간: 다음 주 화요일 오전 10시
아이템: API 리스크 보험 모델 설계
해야 할 일: 
1. API 연결 상태와 SLA 위반 기준 정리
2. 주요 API 분류 및 등급 나누기
3. 보험료 책정 모델 개선 (회귀 분석 잔차 줄이기)
4. 실시간 API 예측 오차 줄이기 위해 구간별 분포 다시 보기
5. 리스크 시나리오 추가 (핀테크 이슈 기반)
팀원들의 의견: 
1. 중소 규모 API에서 데이터 누락 많음
2. 실시간 API에서 예측 오차가 큼
3. 실제 장애 사례 기반으로 리스크 시나리오 만듦
멘토의 피드백: 시나리오가 너무 일반적이라는 피드백을 받았으며, 이를 개선하기 위해 핀테크 쪽 이슈를 기반으로 리스크 요소를 추가하려고 함.
